# 🔧 环境配置和检查

## 概述
本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：
- 使用统一的conda环境
- 通过国内镜像源快速安装依赖
- 加速模型下载
- 检查系统配置

## 配置步骤
1. **Conda环境管理** - 激活统一的学习环境
2. **包管理器优化** - 配置pip使用清华镜像源
3. **模型下载加速** - 设置HuggingFace镜像代理
4. **系统环境诊断** - 检查硬件和软件配置


In [ ]:
# 1. 激活conda环境
%%script bash
# 初始化 conda
eval "$(conda shell.bash hook)"
conda activate flyai_agent_in_action
conda env list


In [ ]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list


In [ ]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

In [ ]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


# Langfuse 提示词管理性能测试

## 测试目标和背景
本笔记本对 Langfuse 提示词管理系统进行性能基准测试，通过测量在无缓存模式下（cache_ttl_seconds=0）连续执行 1,000 次提示词检索和编译操作的延迟时间。

## 关于缓存的重要说明
在实际应用中，这种延迟通常不会成为问题，因为提示词会在 SDK 客户端进行缓存。更多关于缓存机制的信息，请参考 [Langfuse 提示词管理文档](https://langfuse.com/docs/prompt-management/features/caching)。

## 测试结果解读
测试结果包含了网络延迟，因此绝对数值可能会因地理位置和服务器负载而有所不同。建议使用直方图和统计摘要来比较相对性能改进，例如不同 SDK 版本或缓存设置之间的差异。

## 前置条件
测试需要在已认证的项目中设置一个名为 `perf-test` 的提示词模板。

## 适用人群
本教程适合大模型技术初学者，帮助理解提示词管理系统的性能特征和优化方法。

In [ ]:
# 安装 Langfuse Python SDK
# Langfuse 是一个开源的大语言模型应用可观测性平台
# 提供提示词管理、追踪、评估等功能
%pip install langfuse

In [ ]:
import os

# 配置 Langfuse 连接参数
# 这些密钥可以从项目设置页面获取：https://cloud.langfuse.com

# 公钥：用于客户端身份验证，相对安全可以暴露
os.environ["LANGFUSE_PUBLIC_KEY"] = ""

# 私钥：用于服务端身份验证，需要严格保密
os.environ["LANGFUSE_SECRET_KEY"] = ""

# 服务器地址配置
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 欧洲区域服务器
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 美国区域服务器

# 注意：在生产环境中，建议使用 .env 文件或其他安全方式管理这些敏感信息

In [ ]:
# 导入必要的库
import time          # 用于时间测量和延迟控制
import pandas as pd  # 用于数据分析和统计计算
import matplotlib.pyplot as plt  # 用于数据可视化
from tqdm.auto import tqdm       # 用于显示进度条

from langfuse import Langfuse    # Langfuse 主要客户端类

# 从环境变量初始化 Langfuse 客户端
# 这会自动读取之前设置的 LANGFUSE_PUBLIC_KEY、LANGFUSE_SECRET_KEY 和 LANGFUSE_HOST
langfuse = Langfuse()

# 验证客户端是否正确初始化
# 如果认证失败，会抛出断言错误，提示检查环境变量配置
assert langfuse.auth_check(), "Langfuse 客户端初始化失败 - 请检查环境变量配置是否正确"

In [ ]:
# 性能测试配置参数
N_RUNS = 1_000  # 测试运行次数：1000次，足够获得统计学意义的结果
prompt_name = "perf-test"  # 提示词模板名称：需要在 Langfuse 项目中预先创建

# 存储每次操作的执行时间
durations = []

# 执行性能基准测试
for _ in tqdm(range(N_RUNS), desc="性能测试进行中"):
    # 记录开始时间（使用高精度计时器）
    start = time.perf_counter()
    
    # 获取提示词模板（禁用缓存以测试真实网络延迟）
    # cache_ttl_seconds=0 表示不使用缓存，每次都从服务器获取
    prompt = langfuse.get_prompt(prompt_name, cache_ttl_seconds=0)
    
    # 编译提示词（包含服务端处理时间）
    # 使用最小输入进行编译，模拟实际使用场景
    prompt.compile(input="test")
    
    # 计算并记录本次操作的总耗时
    durations.append(time.perf_counter() - start)
    
    # 添加短暂延迟，避免对服务器造成过大压力
    time.sleep(0.05)  # 50毫秒延迟

# 将时间数据转换为 pandas Series 便于后续分析
durations_series = pd.Series(durations, name="执行时间(秒)")

In [ ]:
# 计算详细的统计信息
# percentiles 参数指定要计算的百分位数
# 25%、50%（中位数）、75%、99% 分位数有助于理解数据分布
stats = durations_series.describe(percentiles=[0.25, 0.5, 0.75, 0.99])

# 显示统计结果
# count: 样本数量
# mean: 平均值
# std: 标准差（衡量数据离散程度）
# min/max: 最小值/最大值
# 25%/50%/75%/99%: 对应百分位数的值
stats

## 历史性能测试结果参考

以下是之前的性能测试结果，可以作为对比参考：

```
count    1000.000000  # 测试样本数：1000次
mean        0.039335 sec  # 平均响应时间：约39.3毫秒
std         0.014172 sec  # 标准差：约14.2毫秒（表示数据相对稳定）
min         0.032702 sec  # 最快响应时间：约32.7毫秒
25%         0.035387 sec  # 25%的请求在35.4毫秒内完成
50%         0.037030 sec  # 50%的请求在37.0毫秒内完成（中位数）
75%         0.041111 sec  # 75%的请求在41.1毫秒内完成
99%         0.068914 sec  # 99%的请求在68.9毫秒内完成
max         0.409609 sec  # 最慢响应时间：约409.6毫秒（可能是网络异常）
```

### 结果解读
- **平均响应时间约40毫秒**：对于大多数应用场景来说是可接受的
- **99%的请求在70毫秒内完成**：说明系统性能相对稳定
- **最大值较高**：可能由网络波动或服务器负载引起，属于正常现象

In [ ]:
# 创建性能数据的直方图可视化
plt.figure(figsize=(8,4))  # 设置图表大小：宽8英寸，高4英寸

# 绘制直方图
# bins=30 表示将数据分为30个区间，可以清晰显示数据分布
plt.hist(durations_series, bins=30, alpha=0.7, color='skyblue', edgecolor='black')

# 设置图表标签和标题
plt.xlabel("执行时间 (秒)")  # X轴标签
plt.ylabel("频次")         # Y轴标签
plt.title("Langfuse 提示词管理性能测试 - 响应时间分布")  # 图表标题

# 添加网格线，便于读取数值
plt.grid(True, alpha=0.3)

# 显示图表
plt.show()

# 直方图说明：
# - X轴：响应时间（秒）
# - Y轴：该时间范围内的请求数量
# - 图形形状：通常呈现正态分布或右偏分布
# - 峰值位置：表示最常见的响应时间
# - 长尾：表示少数异常慢的请求

## 历史测试结果图表参考

以下是之前性能测试的可视化结果，展示了响应时间的分布情况：

![性能测试图表](https://langfuse.com/images/docs/prompt-performance-chart.png)

### 图表解读要点
1. **分布形状**：大部分响应时间集中在较小的值附近
2. **峰值位置**：显示最常见的响应时间范围
3. **长尾现象**：少数请求的响应时间较长，这是网络应用的正常现象
4. **性能稳定性**：整体分布相对集中，说明系统性能稳定

### 实际应用建议
- **生产环境**：建议启用缓存（cache_ttl_seconds > 0）以获得更好的性能
- **监控指标**：关注99%分位数，确保绝大多数用户获得良好体验
- **优化策略**：可以通过CDN、地理位置优化等方式进一步提升性能